In [3]:
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)

import pandas as pd
pd.options.display.max_columns = 100

from matplotlib import pyplot as plt
import numpy as np

import seaborn as sns
sns.set(rc={'figure.figsize':(12,9)})
import pylab as plot
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss,accuracy_score
from xgboost import XGBClassifier
import xgboost as xgb

In [4]:
#for scaling
from sklearn.preprocessing import StandardScaler

Importing training dataset

In [5]:
data = pd.read_csv('./train.csv')

In [6]:
print(data.shape)

(867873, 10)


In [7]:
data.columns

Index(['Dates', 'Category', 'Descript', 'DayOfWeek', 'PdDistrict',
       'Resolution', 'Address', 'X', 'Y', 'Id'],
      dtype='object')

Splitting the "Dates" into date and time

In [8]:
df=data
datetime=df.Dates.str.split(pat=" ",expand=True)
datetime.columns=['Date','Time']
#datetime

In [9]:
Date=datetime.Date.str.split(pat="-",expand=True)
Date.columns=['Year','Month','Day']

Time=datetime.Time.str.split(pat=":",expand=True)
Time.columns=['Hour','Minute','Second']

Dropping the column of "Dates" and appending the dataframe "datetime"

In [34]:
df=pd.concat([df,Date,Time],axis=1)
#df

In [35]:
df=df.drop(labels=['Dates'],axis=1)

# #Encoding

In [36]:
df.columns

Index(['Category', 'Descript', 'DayOfWeek', 'PdDistrict', 'Resolution',
       'Address', 'X', 'Y', 'Id', 'Year', 'Month', 'Day', 'Hour', 'Minute',
       'Second'],
      dtype='object')

LabelEncoding of "DayOfWeek" and "PdDistrict"

In [37]:
le = preprocessing.LabelEncoder()

In [38]:
#y=pd.get_dummies(df.Category,columns=['Category'],prefix=" ",prefix_sep=" ",drop_first=True,)

le_res=le.fit_transform(df['Category'])
y=pd.DataFrame(le_res)
y.columns=['Category']
#y

# Address splitting

In [10]:
address=pd.DataFrame(df['Address'],columns=['Address'])
address=address.Address.str.split(pat=" / ",expand=True )
address.columns=['Intr1','Intr2']

In [11]:
address

,Address
0,2100 Block of MISSION ST
1,1000 Block of MARKET ST
2,800 Block of BRYANT ST
3,2400 Block of SAN BRUNO AV
4,4TH ST STEVENSON ST
5,15TH ST NATOMA ST
6,0 Block of MARK LN
7,100 Block of PUTNAM ST
8,HALE ST SAN BRUNO AV
9,ELLIS ST LARKIN ST


In [40]:
df=df.drop(labels=['Address'],axis=1)

In [12]:
temp=address['Intr1'].astype(str).str[-2:]
New_Address=pd.DataFrame(temp)
df=pd.concat([New_Address,df],axis=1)
#df

In [14]:
temp.unique()

array(['ST', 'AV', 'LN', 'BL', 'DR', 'HY', 'CT', 'RD', 'PL', 'PZ', 'WY',
       '80', 'TR', 'AL', 'AY', 'ER', 'AR', 'MS', 'CR', 'TI', 'WK', 'EX',
       '  ', 'RK', 'NO', 'RW'], dtype=object)

In [42]:
Id=df['Id']
df=df.drop(['Descript','Category','Resolution','Id','Minute','Second'],axis=1)

In [43]:
df

,Intr1,DayOfWeek,PdDistrict,X,Y,Year,Month,Day,Hour
0,ST,Friday,MISSION,-122.419331,37.762264,2013,06,28,17
1,ST,Thursday,SOUTHERN,-122.411340,37.781271,2004,02,19,02
2,ST,Wednesday,SOUTHERN,-122.403405,37.775421,2007,11,14,00
3,AV,Thursday,BAYVIEW,-122.404715,37.730161,2007,12,27,18
4,ST,Sunday,SOUTHERN,-122.405239,37.785265,2012,09,09,17
5,ST,Wednesday,MISSION,-122.418405,37.766760,2014,05,14,06
6,LN,Wednesday,CENTRAL,-122.404651,37.790442,2006,08,09,23
7,ST,Friday,INGLESIDE,-122.410871,37.737526,2012,11,16,12
8,ST,Monday,BAYVIEW,-122.405911,37.732964,2007,02,05,23
9,ST,Tuesday,NORTHERN,-122.417710,37.784236,2006,02,07,07


Label Encoding :

In [44]:

le = preprocessing.LabelEncoder()


In [45]:
#le = preprocessing.LabelEncoder()
le_res=le.fit_transform(df['Month'])
Month=pd.DataFrame(le_res)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Month.columns=['Month']
df=df.drop(labels=['Month'],axis=1)
df=pd.concat([Month,df],axis=1)

#le = preprocessing.LabelEncoder()
le_res=le.fit_transform(df['Year'])
Year=pd.DataFrame(le_res)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Year.columns=['Year']
df=df.drop(labels=['Year'],axis=1)
df=pd.concat([Year,df],axis=1)

#le = preprocessing.LabelEncoder()
le_res=le.fit_transform(df['Day'])
Day=pd.DataFrame(le_res)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Day.columns=['Day']
df=df.drop(labels=['Day'],axis=1)
df=pd.concat([Day,df],axis=1)

#le = preprocessing.LabelEncoder()
le_res=le.fit_transform(df['Hour'])
Hour=pd.DataFrame(le_res)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Hour.columns=['Hour']
df=df.drop(labels=['Hour'],axis=1)
df=pd.concat([Hour,df],axis=1)


In [46]:
df.columns

Index(['Hour', 'Day', 'Year', 'Month', 'Intr1', 'DayOfWeek', 'PdDistrict', 'X',
       'Y'],
      dtype='object')

In [47]:
#df.columns=['Address', 'DayOfWeek', 'PdDistrict', 'X', 'Y', 'Year', 'Month', 'Day','Hour']
#le_res
#Day=pd.DataFrame(le_res)
df=pd.get_dummies(df,columns=['Intr1', 'DayOfWeek', 'PdDistrict'],drop_first=True)


In [48]:
df=df.drop(labels=['Intr1_80'],axis=1)
df

,Hour,Day,Year,Month,X,Y,Intr1_AL,Intr1_AR,Intr1_AV,Intr1_AY,Intr1_BL,Intr1_CR,Intr1_CT,Intr1_DR,Intr1_ER,Intr1_EX,Intr1_HY,Intr1_LN,Intr1_MS,Intr1_NO,Intr1_PL,Intr1_PZ,Intr1_RD,Intr1_RK,Intr1_RW,Intr1_ST,Intr1_TI,Intr1_TR,Intr1_WK,Intr1_WY,DayOfWeek_Monday,DayOfWeek_Saturday,DayOfWeek_Sunday,DayOfWeek_Thursday,DayOfWeek_Tuesday,DayOfWeek_Wednesday,PdDistrict_CENTRAL,PdDistrict_INGLESIDE,PdDistrict_MISSION,PdDistrict_NORTHERN,PdDistrict_PARK,PdDistrict_RICHMOND,PdDistrict_SOUTHERN,PdDistrict_TARAVAL,PdDistrict_TENDERLOIN
0,17,27,10,5,-122.419331,37.762264,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,2,18,1,1,-122.411340,37.781271,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
2,0,13,4,10,-122.403405,37.775421,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
3,18,26,4,11,-122.404715,37.730161,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,17,8,9,8,-122.405239,37.785265,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
5,6,13,11,4,-122.418405,37.766760,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0
6,23,8,3,7,-122.404651,37.790442,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0
7,12,15,9,10,-122.410871,37.737526,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
8,23,4,4,1,-122.405911,37.732964,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,7,6,3,1,-122.417710,37.784236,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0


In [49]:
data['PdDistrict'].unique()

array(['MISSION', 'SOUTHERN', 'BAYVIEW', 'CENTRAL', 'INGLESIDE',
       'NORTHERN', 'RICHMOND', 'TARAVAL', 'TENDERLOIN', 'PARK'],
      dtype=object)

In [50]:
df.columns

Index(['Hour', 'Day', 'Year', 'Month', 'X', 'Y', 'Intr1_AL', 'Intr1_AR',
       'Intr1_AV', 'Intr1_AY', 'Intr1_BL', 'Intr1_CR', 'Intr1_CT', 'Intr1_DR',
       'Intr1_ER', 'Intr1_EX', 'Intr1_HY', 'Intr1_LN', 'Intr1_MS', 'Intr1_NO',
       'Intr1_PL', 'Intr1_PZ', 'Intr1_RD', 'Intr1_RK', 'Intr1_RW', 'Intr1_ST',
       'Intr1_TI', 'Intr1_TR', 'Intr1_WK', 'Intr1_WY', 'DayOfWeek_Monday',
       'DayOfWeek_Saturday', 'DayOfWeek_Sunday', 'DayOfWeek_Thursday',
       'DayOfWeek_Tuesday', 'DayOfWeek_Wednesday', 'PdDistrict_CENTRAL',
       'PdDistrict_INGLESIDE', 'PdDistrict_MISSION', 'PdDistrict_NORTHERN',
       'PdDistrict_PARK', 'PdDistrict_RICHMOND', 'PdDistrict_SOUTHERN',
       'PdDistrict_TARAVAL', 'PdDistrict_TENDERLOIN'],
      dtype='object')

Label Encoding: PdDistrict

36 Crime Category are there 

# Applying different Models

In [51]:
#Independent Column
X=df
X.shape

(867873, 45)

In [52]:
#Dependent column
y.shape

(867873, 1)

In [53]:
#splitting the dataset into the training set and test set
#from sklearn import model_selection as ms

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0)

In [54]:
"""from sklearn.preprocessing import StandardScaler
sc_X=StandardScaler()
X_train=sc_X.fit_tranform(X_train)
X_test=sc_X.transform(X_test)"""

'from sklearn.preprocessing import StandardScaler\nsc_X=StandardScaler()\nX_train=sc_X.fit_tranform(X_train)\nX_test=sc_X.transform(X_test)'

# Logistic Regression

Feature scaling


In [98]:
from sklearn.linear_model import LogisticRegression
#weight={Address:3,District:3,X:1,Day:2}
#weight={LARCENY/THEFT:35}
classifier = LogisticRegression(penalty='l2',random_state = 0,class_weight='balanced',multi_class='multinomial', solver='lbfgs',n_jobs=-1)
classifier.fit(X_train,y_train)
# Applying k-Fold Cross Validation
"""from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X=df , y=y , cv = 10,n_jobs=-1)
print("Logistic Regression:\n Accuracy:", accuracies.mean(), "+/-", accuracies.std(),"\n")
"""

'from sklearn.model_selection import cross_val_score\naccuracies = cross_val_score(estimator = classifier, X=df , y=y , cv = 10,n_jobs=-1)\nprint("Logistic Regression:\n Accuracy:", accuracies.mean(), "+/-", accuracies.std(),"\n")\n'

In [99]:
#predicting the Test set result
y_pred=classifier.predict_proba(X_test)

In [100]:
# y_test=np.array(y_pred)
# y_pred

In [101]:
type(y_test)

pandas.core.frame.DataFrame

In [102]:
X_train.columns


Index(['Hour', 'Day', 'Month', 'Year', 'Address', 'District', 'DayOfWeek', 'X',
       'Y'],
      dtype='object')

In [103]:
# def logloss(y_true, y_pred, eps=1e-15):
#     y_pred = np.clip(y_pred, eps, 1 - eps)
#     return -(y_true * np.log(y_pred)).sum(axis=1).mean()
ll = log_loss(y_test,y_pred)
ll

3.5351830522637937

# XGBoost Model

In [104]:
X.head()

,Hour,Day,Month,Year,Address,District,DayOfWeek,X,Y
0,17,27,5,10,21,3,0,-122.419331,37.762264
1,2,18,1,1,21,7,4,-122.411340,37.781271
2,0,13,10,4,21,7,6,-122.403405,37.775421
3,18,26,11,4,4,0,4,-122.404715,37.730161
4,17,8,8,9,21,7,3,-122.405239,37.785265


In [108]:
model = xgb.XGBClassifier(objective='multi:softprob')
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [109]:
y_pred=model.predict_proba(X_test)

In [110]:
print (log_loss(y_test,y_pred));

2.4825743596979577


# KNN  

In [39]:
#libraries
from sklearn.neighbors import KNeighborsClassifier

In [64]:
neigh = KNeighborsClassifier(n_neighbors =36,weights='distance',algorithm='ball_tree',n_jobs=-1,p=1, metric = 'minkowski',leaf_size=60)
neigh.fit(X_train,y_train)


C:\anaconda\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


KNeighborsClassifier(algorithm='ball_tree', leaf_size=60, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=36, p=1,
           weights='distance')

In [65]:
y_pred=neigh.predict_proba(X_test)

In [66]:
print (log_loss(y_test,y_pred));

11.481771132230303


ROC curve

In [63]:
"""from sklearn import metrics as ms
ms.roc_curve(y_test,y_pred)"""


'from sklearn import metrics as ms\nms.roc_curve(y_test,y_pred)'

# Random Forest

In [56]:
from sklearn.ensemble import RandomForestClassifier

In [63]:
clf = RandomForestClassifier(n_estimators = 10,criterion = 'gini',random_state = 0)
clf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [64]:
y_pred=clf.predict_proba(X_test)

In [65]:
print (log_loss(y_test,y_pred));

14.175824849717799


In [86]:
df

,X,Y,Intr1_AL,Intr1_AR,Intr1_AV,Intr1_AY,Intr1_BL,Intr1_CR,Intr1_CT,Intr1_DR,Intr1_ER,Intr1_EX,Intr1_HY,Intr1_LN,Intr1_MS,Intr1_NO,Intr1_PL,Intr1_PZ,Intr1_RD,Intr1_RK,Intr1_RW,Intr1_ST,Intr1_TI,Intr1_TR,Intr1_WK,Intr1_WY,DayOfWeek_Monday,DayOfWeek_Saturday,DayOfWeek_Sunday,DayOfWeek_Thursday,DayOfWeek_Tuesday,DayOfWeek_Wednesday,PdDistrict_CENTRAL,PdDistrict_INGLESIDE,PdDistrict_MISSION,PdDistrict_NORTHERN,PdDistrict_PARK,PdDistrict_RICHMOND,PdDistrict_SOUTHERN,PdDistrict_TARAVAL,PdDistrict_TENDERLOIN,Year_2004,Year_2005,Year_2006,Year_2007,Year_2008,Year_2009,Year_2010,Year_2011,Year_2012,...,Day_05,Day_06,Day_07,Day_08,Day_09,Day_10,Day_11,Day_12,Day_13,Day_14,Day_15,Day_16,Day_17,Day_18,Day_19,Day_20,Day_21,Day_22,Day_23,Day_24,Day_25,Day_26,Day_27,Day_28,Day_29,Day_30,Day_31,Hour_01,Hour_02,Hour_03,Hour_04,Hour_05,Hour_06,Hour_07,Hour_08,Hour_09,Hour_10,Hour_11,Hour_12,Hour_13,Hour_14,Hour_15,Hour_16,Hour_17,Hour_18,Hour_19,Hour_20,Hour_21,Hour_22,Hour_23
0,-122.419331,37.762264,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,-122.411340,37.781271,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,-122.403405,37.775421,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,-122.404715,37.730161,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,-122.405239,37.785265,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
5,-122.418405,37.766760,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,-122.404651,37.790442,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
7,-122.410871,37.737526,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
8,-122.405911,37.732964,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
9,-122.417710,37.784236,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# Naive Bayes

In [55]:
from sklearn.naive_bayes import GaussianNB
nv = GaussianNB(priors=None, var_smoothing=1e-01)
nv.fit(X_train,y_train)
y_pred=nv.predict_proba(X_test)
print (log_loss(y_test,y_pred));

2.655355382409067


In [72]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = nv, X=y_train , y=y_test , cv = 10)

ValueError: Found input variables with inconsistent numbers of samples: [607511, 260362]

In [71]:

print(accuracies)

[0.20129745 0.20005375 0.19824144 0.19909381 0.19980031 0.19794907
 0.20144438 0.19825566 0.20048421 0.19771688]
